# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Damien Rupp

Date: 8/19/25

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [1]:
# !pip install pandas
# !pip install yfinance

import os, pathlib, datetime as dt
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from dotenv import load_dotenv

RAW = pathlib.Path('data/raw'); RAW.mkdir(parents=True, exist_ok=True)
load_dotenv(); print('ALPHAVANTAGE_API_KEY loaded?', bool(os.getenv('ALPHAVANTAGE_API_KEY')))

ALPHAVANTAGE_API_KEY loaded? True


## Helpers (use or modify)

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

# print(ts())


def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path


def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}


    
# df = pd.DataFrame(
#     np.random.randint(0, 100, (2, 10)),
#     index=["row1", "row2"],
#     columns=[f"col{i}" for i in range(1, 11)])

# save_csv(df, "dummy_data", year=2025, meaning="Nothing")

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [3]:
SYMBOL = 'CHFUSD=X'

# I could not get ALPHAVANTAGE to get me anything with my key
USE_ALPHA = False #bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY_ADJUSTED','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. adjusted close':'adj_close'})[['date','adj_close']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['adj_close'] = pd.to_numeric(df_api['adj_close'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='9mo', interval='1d').reset_index()[["Date", "Close"]]

    #df_api.columns = ['date','adj_close']
    #print(df_api.info())

print(df_api.head())
v_api = validate(df_api, ['Date','Close']); v_api

C:\Users\damie\AppData\Local\Temp\ipykernel_15936\4211626376.py:16: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_api = yf.download(SYMBOL, period='9mo', interval='1d').reset_index()[["Date", "Close"]]
[*********************100%***********************]  1 of 1 completed

Price        Date     Close
Ticker             CHFUSD=X
0      2024-11-19  1.132246
1      2024-11-20  1.133697
2      2024-11-21  1.131670
3      2024-11-22  1.127523
4      2024-11-25  1.123002


{'missing': [], 'shape': (193, 2), 'na_total': 0}

In [41]:
_ = save_csv(df_api.sort_values('Date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data\raw\api_source-yfinance_symbol-CHFUSD=X_20250819-182505.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [17]:
SCRAPE_URL = 'https://www.formula1.com/en/results/2025/team'  # TODO: replace with permitted page
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (10, 3), 'na_total': 0}

In [18]:
_ = save_csv(df_scrape, prefix='scrape', site='formula1-com', table='team-standing-2025')

Saved data\raw\scrape_site-formula1-com_table-team-standing-2025_20250819-192026.csv


## Documentation
### API Source: (URL/endpoint/params)

I could not use the ALPHAVANTAGE API so I used yFinance

### Scrape Source: (URL/table description)

https://www.iban.com/currency-codes : This table contains the country name, the currency, the code, and the number in the ibans

https://en.wikipedia.org/wiki/List_of_S%26P_500_companies : This table contains a lot of information about the fortune 500 companies, including their names, the location of their headquarters, their sector...

https://www.formula1.com/en/results/2025/team : This table contains the name of each F1 team, the point they have so far in the season and their position in the championship.

### Assumptions & risks: (rate limits, selector fragility, schema changes)

The website changing the name or the kind of information uploaded.

### Confirm `.env` is not committed.

I have committed a .env.example, but the one with my own key is stored on my computer